# So you want to convert Tensorflow to MxNet, eh? 

Lets try parsing the API's and generate a mapping between the two. This might take a while

In [1]:
import mxnet as mx
import tensorflow as tf
import pkgutil, types, pprint
import numpy as np

def doc_dump(base, docs={}):
    try:
        for a in dir(eval(base)):
            if isinstance(eval(base).__dict__.get(a), types.FunctionType):
                doc = eval(base).__dict__.get(a).__doc__
                docs[base + '.' + a] = doc if doc else ''
        for importer, modname, ispkg in pkgutil.walk_packages(path=eval(base).__path__,
                                                              prefix=base + '.',
                                                              onerror=lambda e: None):
            docs = doc_dump(modname, docs)
    except Exception as e:
        pass
    return docs
        
tf_docs = doc_dump('tf', {})
mx_docs = doc_dump('mx', {})

/usr/local/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/usr/local/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))


We will just do an n-gram model for clustering. we will use 3-grams

In [2]:
TOTAL_FEATURES = 1000
N_GRAM_WIDTHS = [2, 3, 4]

def tfidf(str_list, ngram_width=3):
    tf = {}
    for s in str_list:
        for start, end in zip(range(len(s) - ngram_width),
                              range(ngram_width, len(s))):
            if s[start:end] not in tf:
                tf[s[start:end]] = 0
            tf[s[start:end]] += 1
    
    idf = {}
    for t in tf.keys():
        cnt = 0
        for s in str_list:
            if t in s:
                cnt += 1
            idf[t] = len(str_list)/(cnt + 1.0)
    
    return {t:tf[t]*idf[t] for t in tf.keys()}

ranks = {}
for N in N_GRAM_WIDTHS:
    ranks.update(tfidf(list(tf_docs.values()) + list(mx_docs.values()), ngram_width=N))
ngram_keys = [t[1] for t in sorted([(v, k) for k, v in ranks.items()])[-TOTAL_FEATURES:]]

In [8]:
tf_labels, tf_values = zip(*list(tf_docs.items()))
mx_labels, mx_values = zip(*list(mx_docs.items()))

tf_data = mx.nd.array([[s.count(k)
                        for k in ngram_keys]
                       for s in tf_values]).tostype('csr')
mx_data = mx.nd.array([[s.count(k)
                        for k in ngram_keys]
                       for s in mx_values]).tostype('csr')

In [28]:
cos_dist = mx.nd.dot(tf_data, mx_data.T)/mx.nd.sqrt(mx.nd.sum(tf_data * tf_data, axis=1).reshape((-1, 1)) * 
                                                    mx.nd.sum(mx_data * mx_data, axis=1).reshape((1, -1)))
cos_dist = np.nan_to_num(cos_dist.asnumpy())
mu, sig = np.mean(cos_dist), np.std(cos_dist)
print(mu, sig)
mapping = {}
for i, tf_label in enumerate(tf_labels):
    inds = np.argsort(cos_dist[i, :])
    if cos_dist[i, inds[-4]] > mu + 1.7*sig:
        mapping[tf_label] = list(reversed([mx_labels[j] for j in inds[-5:]]))
pprint.pprint(mapping)

0.564833 0.226674
{'tf.contrib.grid_rnn.namedtuple': ['mx.recordio.namedtuple',
                                    'mx.io.namedtuple',
                                    'mx.model.namedtuple',
                                    'mx.test_utils.contextmanager',
                                    'mx.recordio.unpack_img'],
 'tf.contrib.grid_rnn.python.ops.grid_rnn_cell.namedtuple': ['mx.recordio.namedtuple',
                                                             'mx.io.namedtuple',
                                                             'mx.model.namedtuple',
                                                             'mx.test_utils.contextmanager',
                                                             'mx.recordio.unpack_img'],
 'tf.contrib.keras.api.keras.backend.eval': ['mx.ndarray.ndarray.power',
                                             'mx.ndarray.power',
                                             'mx.ndarray.ndarray.modulo',
                             